In [1]:
import numpy as np

shape = (100, 5)
n = shape[1]

np.random.seed(10)
X = np.random.randint(-5,6, size=shape) # Integers from -5 to 5
y = np.sum(X, axis=1) # Sum of each row
X = X + np.random.normal(0, .1, shape) # Add noise to input


In [11]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y)
reg.score(X, y), reg.coef_

(0.9991058582387404,
 array([0.99320348, 0.99542534, 0.99506914, 1.00549516, 0.98140095]))

In [27]:
def divisive_shap_approx(X, y, model, beta, gamma_n, gamma_d):
    if X.shape[1] > np.emath.logn(beta, n):
        split_S = partition(X,y,model)
        gamma_n = gamma_n*value(X, y, model)
        gamma_d = gamma_d*sum([value(Xi, yi, model) for Xi, yi in split_S])
        return union([divisive_shap_approx(Xi, 
                                           yi, 
                                           model, 
                                           gamma_n, 
                                           gamma_d) for Xi, yi in split_S])
    else:
        shap = shapley_true(X, y, model)
        shap = shap*gamma_n/gamma_d
        return (X, y, shap)



def value(X, y, model):
    reg = model.fit(X, y)
    return reg.score(X, y)
        

def shapley_true(X, y, model):
    """Returns true shapley value of each feature (numpy array)"""
    pass

def partition(X, y, model):
    """Returns list of tuples: [(Xi, Yi, Shapi), ...]"""
    N = X.shape[1]
    return [(X[:,:N//2], y), (X[:,N//2:], y)]

def union(S_split):
    """Returns the union of several sets (Xi, Yi, Shapi)"""
    X = S_split[0][0]
    y = S_split[0][1]
    shap = S_split[0][2]
    for Xi, _, shapi in S_split[1:]:
        X = np.concatenate((X, Xi), axis=1)
        shap = np.concatenate((shap, shapi))
    return (X, y, shap)
